# Import Libraries

In [34]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import chromadb
import os

load_dotenv()

True

_____

# Setup LLM

In [35]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=1,
    groq_api_key=os.getenv("GROQ_API_KEY")  
)

_________________

# Job Description in JSON

In [36]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/senior-ai-ml-engineer/job/R-66196")
page_data = loader.load().pop().page_content

page_data = "\n".join(
    line.strip() for line in page_data.splitlines() if line.strip()
)

print(page_data)

Senior AI/ML Engineer
Skip to main content
Open Virtual Assistant
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Language
Select a Language
Deutsch
English
Español (España)
Español (América Latina)
Français
Italiano
Nederlands
Polski
Tiếng Việt
Türkçe
简体中文
繁體中文
עִברִית
한국어
日本語
Careers
Close Menu
Careers
Chat
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Jordan Careers
Converse Careers
Language
Menu
Return to Previous Menu
Select a Language
Deutsch
English
Español (España)
Español (América Latina)
Français
Italiano
Nederlands
Polski
Tiếng Việt
Türkçe
简体中文
繁體中文
עִברִית
한국어
日本語
Back to Search
Senior AI/ML Engineer
Categories ID
Categories URL
Position Type
Full Time
Date Posted
Primary Quest ID
Second Quest ID
Job Classification
Career area
Software Engineering
Location
Remote
Job ID
R-66196
Apply Now
Share Job
Share Job Posting
Facebook
Opens In A New Tab
LinkedIn
Link Opens In New Window
Email
Close-Medium (Default Size)-icon
Close Menu
Become a Part of the NIKE, Inc. Team
NIKE, 

In [37]:
from langchain.prompts import PromptTemplate

prompt_extract = PromptTemplate(
    input_variables=["page_data"],
    template="""
### Scraped Text from Web url
{page_data}

# Instruction
The scraped text is from the career's page of a website.
Your job is to extract the job postings and return them in JSON format containing the 
following keys: `role`, `experience`, `skills in detail`, and `description`. 
Only return the valid JSON.

### VALID JSON (NO PREAMBLE):
"""
)

In [38]:
chain_extract = prompt_extract | llm 
job_description = chain_extract.invoke(input={'page_data':page_data})
print(f"type: {type(job_description.content)})")

type: <class 'str'>)


In [39]:
from langchain_core.output_parsers import JsonOutputParser
from pprint import pprint

json_parser = JsonOutputParser()
job_description = json_parser.parse(job_description.content)
pprint(job_description)
print(f"\ntype: {type(job_description)})")

{'description': 'We seek passionate engineers to join our team. As a Senior '
                'AI/ML Engineer, you will influence and develop robust machine '
                'learning and generative AI solutions that have a direct '
                'impact on the business. You will be working with the '
                'Corporate Functions Artificial Intelligence team at Nike '
                'focused on delivering AI capabilities for Nike’s corporate '
                'functions.',
 'experience': '5 to 7 years of professional experience in software '
               'engineering',
 'role': 'Senior AI/ML Engineer',
 'skills in detail': ['Python',
                      'algorithms and data structures',
                      'AWS experience',
                      'database technology (e.g. Postgres, Redis)',
                      'data processing technology (e.g. Sagemaker or '
                      'DataBricks)',
                      'MLOps',
                      'Scikit-learn',
   

___________________

# Resume in Json

In [40]:
with open("my_resume.txt", "r", encoding="utf-8") as f:
    resume = f.read()

resume_to_json_prompt = PromptTemplate(
    input_variables=["resume"],
    template="""
You are an assistant that converts resumes into structured JSON.
Each section or subheading in the resume should become a key.
The content under each heading should become the value.
Preserve all important details.

Resume Text:
{resume}

Return only valid JSON without any extra text.
"""
)

In [41]:
full_prompt = resume_to_json_prompt.format(resume=resume)

res = llm.invoke(full_prompt)  

json_parser = JsonOutputParser()
resume = json_parser.parse(res.content)

resume

{'Contact': {'Name': 'Rohit Karthick', 'Location': 'Chennai, Tamil Nadu, India', 'Phone': '+91-9566887888', 'Email': 'rkbeo5@gmail.com', 'LinkedIn': 'https://www.linkedin.com/in/rohit-karthick/', 'Github': 'https://github.com/7rohxt'}, 'Technical Skills': {'Languages': ['Python', 'SQL'], 'ML/DL': ['TensorFlow', 'Keras', 'Scikit-learn', 'OpenCV', 'Transformers'], 'Web/API': ['FastAPI', 'Flask', 'Streamlit'], 'DevOps': ['Docker', 'AWS EC2', 'Firebase', 'Git'], 'LLM Tools': ['LangChain', 'LoRA fine-tuning']}, 'Professional Experience': [{'Company': 'NeubAItics Pvt. Tech. Ltd.', 'Location': 'Chennai, India', 'Roles': [{'Title': 'Machine Learning Intern', 'Duration': 'Jan 2025 to Jun 2025', 'Achievements': ['Assisted in building a real-time person tracking system using DeepSORT and InsightFace across zones and multiple CCTV cameras.', 'Built and dockerized a real-time face attendance system with spoof detection using OpenCV and FastAPI; integrated Firebase for secure image storage and times

_________________

# Generate Email with Zeroshot

In [42]:
email_prompt = f"""
You are the candidate described in the resume.
Job description: {job_description}
Candidate resume: {resume}

Write a concise, professional cold email to the hiring manager that clearly explains why the candidate is a strong fit for this specific role. 

Instructions:
- Highlight how the candidate's skills, frameworks, and technologies match the job requirements exactly.
- Connect relevant projects or experience to each key requirement from the job description.
- If the candidate does not meet the required years of experience, emphasize related projects and achievements.
- Keep it short, persuasive, and directly aligned with the job posting.
- Avoid generic statements; make it specific to the skills, frameworks, and technologies listed in the job description.
"""

email_response = llm.invoke(email_prompt)

print(email_response.content)

Subject: Application for Senior AI/ML Engineer Role at Nike

Dear Hiring Manager,

I'm excited to apply for the Senior AI/ML Engineer position at Nike, where I can leverage my skills in machine learning, cloud architecture, and data engineering to drive business impact. With a strong foundation in Python, algorithms, and data structures, I'm confident in my ability to develop robust AI solutions.

My experience with TensorFlow, Scikit-learn, and PyTorch aligns with the job requirements, and I've demonstrated proficiency in these frameworks through projects like the LoRA Fine-Tuning Project and Heart Sound Anomaly Detection using Conv1D and LSTM. I've also worked with FastAPI, Docker, and AWS EC2, which are essential for cloud architecture and deployment.

As a Machine Learning Engineer at NeubAItics Pvt. Tech. Ltd., I've developed real-time systems using OpenCV, FastAPI, and Docker, showcasing my ability to work with complex data sets and ETL pipelines. My project, Fake Reviews Classif

__________________

# Vector Database

In [43]:
from few_shots import few_shots

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [44]:
chroma_client = chromadb.PersistentClient(path="chroma_db") 

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

collection = chroma_client.get_or_create_collection(name="few_shots")

### Retrive one example with most similar Job description

In [45]:
import json

for i, example in enumerate(few_shots):
    job_text = json.dumps(example["job_description"], indent=2)
    collection.add(
        ids=[f"example_{i}"],
        documents=[job_text],
        metadatas=[{"resume": json.dumps(example["resume"]), "email": example["email"]}]
    )

In [56]:
query = "AI/ML Engineer with AWS and Python"

results = collection.query(
    query_texts=[query],
    n_results=1
)

print("Top match:\n", results)

Top match:
 {'ids': [['example_1']], 'embeddings': None, 'documents': [['{\n  "role": "AI/ML Engineer",\n  "experience": "2-5 years of professional experience in software engineering",\n  "skills in detail": [\n    "Python",\n    "Algorithms and data structures",\n    "AWS experience",\n    "Database technology (e.g. Postgres, Redis)",\n    "Data processing technology (e.g. Sagemaker or DataBricks)",\n    "MLOps",\n    "Cloud architecture and technologies, especially Amazon Web Services: ECR, SageMaker, Lambda, API Gateway",\n    "Frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI",\n    "Pipeline orchestration tools such as AirFlow or Databricks Workflows",\n    "Spark, Kubernetes, Docker, Jenkins, Databricks, or Terraform"\n  ],\n  "description": "Influence and develop robust machine learning and generative AI solutions that have a direct impact on the business, designing and implementing scalable applications that leverage prediction models and optimization program